In [45]:
import cirq
import numpy as np
import matplotlib.pyplot as plt

#3 qubit x rotation
def rotation3Z(q0, q1, q2, angle):
    a = cirq.NamedQubit("A")
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2) #Needed for superposition
    yield cirq.CNOT(q0, a)
    yield cirq.CNOT(q1, a)
    yield cirq.CNOT(q2, a)
    yield cirq.rz(2 * angle)(a)
    yield cirq.CNOT(q2, a)
    yield cirq.CNOT(q1, a)
    yield cirq.CNOT(q0, a)
    #yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
    
#Finds the expectation value of a measurement in the z basis
def expectation3Z(circuit, mqbits):
    sim = cirq.Simulator()
    circuit.append(cirq.measure(*mqbits, key='z'))
    print(circuit)
    
    counts = 1000
    samples = sim.run(circuit, repetitions=counts)
    data = samples.histogram(key='z')
    
    binaryformat = "{0:0" + str(3) + "b}"
    
    z3obj = 0
    
    #Dictionary: key = qubit state measurement, value = number of results
    for key, value in data.items():
        bitstring = binaryformat.format(int(key))#Convert to binary to get the state
        z3obj += int(value) * (-1)**(int(bitstring[0])+int(bitstring[1])+int(bitstring[2]))
        
    z3obj /= counts
    return z3obj

In [46]:
q = cirq.LineQubit.range(3)
circuit = cirq.Circuit(rotation3Z(q[0],q[1],q[2],np.pi/5))
q2 = [q[0], q[1], q[2]]
print(expectation3Z(circuit, q2))

0: ───H───@──────────────────────────────@───M('z')───
          │                              │   │
1: ───H───┼───@──────────────────────@───┼───M────────
          │   │                      │   │   │
2: ───H───┼───┼───@──────────────@───┼───┼───M────────
          │   │   │              │   │   │
A: ───────X───X───X───Rz(0.4π)───X───X───X────────────
-0.006


In [47]:
q = cirq.LineQubit.range(3)
circuit = cirq.Circuit(rotation3Z(q[0],q[1],q[2],np.pi/9))

#Measure each qubit individually
circuit.append(cirq.measure(q[0],key='q0'))
circuit.append(cirq.measure(q[1],key='q1'))
circuit.append(cirq.measure(q[2],key='q2'))
print(circuit)

sim = cirq.Simulator()
result = sim.run(circuit, repetitions = 1000)
print(result.histogram(key='q0'))
print(result.histogram(key='q1'))
print(result.histogram(key='q2'))

#THIS IS AN ENTANGLING GATE
#Of course the z expectation value is always 1
#Adding x rotations on after the entanglement rotation will shift the qubits slightly out of alignment

                                       ┌────────┐   ┌────────┐
0: ───H───@──────────────────────────────────────────@───────────M('q0')───
          │                                          │
1: ───H───┼───@─────────────────────────@────────────┼M('q1')──────────────
          │   │                         │            │
2: ───H───┼───┼───@────────────────@────┼M('q2')─────┼─────────────────────
          │   │   │                │    │            │
A: ───────X───X───X───Rz(0.222π)───X────X────────────X─────────────────────
                                       └────────┘   └────────┘
Counter({0: 512, 1: 488})
Counter({1: 521, 0: 479})
Counter({0: 524, 1: 476})


In [44]:
q = cirq.LineQubit.range(3)
circuit = cirq.Circuit(rotation3Z(q[0],q[1],q[2],np.pi/32))
circuit.append(cirq.rx(np.pi/2)(q[2]))
circuit.append(cirq.rx(-np.pi/8)(q[1]))

q2 = [q[0], q[1], q[2]]
print(expectation3Z(circuit, q2))

                                       ┌─────────┐   ┌────────────┐
0: ───H───@───────────────────────────────────────────@───────────────M('z')───
          │                                           │               │
1: ───H───┼───@─────────────────────────@─────────────┼Rx(-0.125π)────M────────
          │   │                         │             │               │
2: ───H───┼───┼───@────────────────@────┼Rx(0.5π)─────┼───────────────M────────
          │   │   │                │    │             │
A: ───────X───X───X───Rz(0.062π)───X────X─────────────X────────────────────────
                                       └─────────┘   └────────────┘
0.17
